<a href="https://colab.research.google.com/github/quyettranvu/deep_learning_hands_on/blob/feature/chapter_builders-guide/model-construction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Layers and Modules
:label:`sec_model_construction`

When we first introduced neural networks,
we focused on linear models with a single output.
Here, the entire model consists of just a single neuron.
Note that a single neuron
(i) takes some set of inputs;
(ii) generates a corresponding scalar output;
and (iii) has a set of associated parameters that can be updated
to optimize some objective function of interest.
Then, once we started thinking about networks with multiple outputs,
we leveraged vectorized arithmetic
to characterize an entire layer of neurons.
Just like individual neurons,
layers (i) take a set of inputs,
(ii) generate corresponding outputs,
and (iii) are described by a set of tunable parameters.
When we worked through softmax regression,
a single layer was itself the model.
However, even when we subsequently
introduced MLPs,
we could still think of the model as
retaining this same basic structure.

Interestingly, for MLPs,
both the entire model and its constituent layers
share this structure.
The entire model takes in raw inputs (the features),
generates outputs (the predictions),
and possesses parameters
(the combined parameters from all constituent layers).
Likewise, each individual layer ingests inputs
(supplied by the previous layer)
generates outputs (the inputs to the subsequent layer),
and possesses a set of tunable parameters that are updated
according to the signal that flows backwards
from the subsequent layer.


While you might think that neurons, layers, and models
give us enough abstractions to go about our business,
it turns out that we often find it convenient
to speak about components that are
larger than an individual layer
but smaller than the entire model.
For example, the ResNet-152 architecture,
which is wildly popular in computer vision,
possesses hundreds of layers.
These layers consist of repeating patterns of *groups of layers*. Implementing such a network one layer at a time can grow tedious.
This concern is not just hypothetical---such
design patterns are common in practice.
The ResNet architecture mentioned above
won the 2015 ImageNet and COCO computer vision competitions
for both recognition and detection :cite:`He.Zhang.Ren.ea.2016`
and remains a go-to architecture for many vision tasks.
Similar architectures in which layers are arranged
in various repeating patterns
are now ubiquitous in other domains,
including natural language processing and speech.

To implement these complex networks,
we introduce the concept of a neural network *module*.
A module could describe a single layer,
a component consisting of multiple layers,
or the entire model itself!
One benefit of working with the module abstraction
is that they can be combined into larger artifacts,
often recursively. This is illustrated in :numref:`fig_blocks`. By defining code to generate modules
of arbitrary complexity on demand,
we can write surprisingly compact code
and still implement complex neural networks.

![Multiple layers are combined into modules, forming repeating patterns of larger models.](http://d2l.ai/_images/blocks.svg)
:label:`fig_blocks`


From a programming standpoint, a module is represented by a *class*.
Any subclass of it must define a forward propagation method
that transforms its input into output
and must store any necessary parameters.
Note that some modules do not require any parameters at all.
Finally a module must possess a backpropagation method,
for purposes of calculating gradients.
Fortunately, due to some behind-the-scenes magic
supplied by the auto differentiation
(introduced in :numref:`sec_autograd`)
when defining our own module,
we only need to worry about parameters
and the forward propagation method.


In [1]:
import torch
from torch import nn
from torch.nn import functional as F

[**To begin, we revisit the code
that we used to implement MLPs**]
(:numref:`sec_mlp`).
The following code generates a network
with one fully connected hidden layer
with 256 units and ReLU activation,
followed by a fully connected output layer
with ten units (no activation function).


In [2]:
net = nn.Sequential(nn.LazyLinear(256), nn.ReLU(), nn.LazyLinear(10))

X = torch.rand(2, 20)
net(X).shape

torch.Size([2, 10])

In this example, we constructed
our model by instantiating an `nn.Sequential`, with layers in the order
that they should be executed passed as arguments.
In short, (**`nn.Sequential` defines a special kind of `Module`**),
the class that presents a module in PyTorch.
It maintains an ordered list of constituent `Module`s.
Note that each of the two fully connected layers is an instance of the `Linear` class
which is itself a subclass of `Module`.
The forward propagation (`forward`) method is also remarkably simple:
it chains each module in the list together,
passing the output of each as input to the next.
Note that until now, we have been invoking our models
via the construction `net(X)` to obtain their outputs.
This is actually just shorthand for `net.__call__(X)`.


## [**A Custom Module**]

Perhaps the easiest way to develop intuition
about how a module works
is to implement one ourselves.
Before we do that,
we briefly summarize the basic functionality
that each module must provide:


1. Ingest input data as arguments to its forward propagation method.
1. Generate an output by having the forward propagation method return a value. Note that the output may have a different shape from the input. For example, the first fully connected layer in our model above ingests an input of arbitrary dimension but returns an output of dimension 256.
1. Calculate the gradient of its output with respect to its input, which can be accessed via its backpropagation method. Typically this happens automatically.
1. Store and provide access to those parameters necessary
   for executing the forward propagation computation.
1. Initialize model parameters as needed.


In the following snippet,
we code up a module from scratch
corresponding to an MLP
with one hidden layer with 256 hidden units,
and a 10-dimensional output layer.
Note that the `MLP` class below inherits the class that represents a module.
We will heavily rely on the parent class's methods,
supplying only our own constructor (the `__init__` method in Python) and the forward propagation method.


In [3]:
class MLP(nn.Module):
    def __init__(self):
        # Call the constructor of the parent class nn.Module to perform
        # the necessary initialization
        super().__init__()
        self.hidden = nn.LazyLinear(256)
        self.out = nn.LazyLinear(10)

    # Define the forward propagation of the model, that is, how to return the
    # required model output based on the input X
    def forward(self, X):
        return self.out(F.relu(self.hidden(X)))

Let's first focus on the forward propagation method.
Note that it takes `X` as input,
calculates the hidden representation
with the activation function applied,
and outputs its logits.
In this `MLP` implementation,
both layers are instance variables.
To see why this is reasonable, imagine
instantiating two MLPs, `net1` and `net2`,
and training them on different data.
Naturally, we would expect them
to represent two different learned models.

We [**instantiate the MLP's layers**]
in the constructor
(**and subsequently invoke these layers**)
on each call to the forward propagation method.
Note a few key details.
First, our customized `__init__` method
invokes the parent class's `__init__` method
via `super().__init__()`
sparing us the pain of restating
boilerplate code applicable to most modules.
We then instantiate our two fully connected layers,
assigning them to `self.hidden` and `self.out`.
Note that unless we implement a new layer,
we need not worry about the backpropagation method
or parameter initialization.
The system will generate these methods automatically.
Let's try this out.


In [4]:
net = MLP()
net(X).shape

torch.Size([2, 10])

A key virtue of the module abstraction is its versatility.
We can subclass a module to create layers
(such as the fully connected layer class),
entire models (such as the `MLP` class above),
or various components of intermediate complexity.
We exploit this versatility
throughout the coming chapters,
such as when addressing
convolutional neural networks.


## [**The Sequential Module**]
:label:`subsec_model-construction-sequential`

We can now take a closer look
at how the `Sequential` class works.
Recall that `Sequential` was designed
to daisy-chain other modules together.
To build our own simplified `MySequential`,
we just need to define two key methods:

1. A method for appending modules one by one to a list.
1. A forward propagation method for passing an input through the chain of modules, in the same order as they were appended.

The following `MySequential` class delivers the same
functionality of the default `Sequential` class.


In [6]:
class MySequential(nn.Module):
    def __init__(self, *args):
        super().__init__()
        for idx, module in enumerate(args):
            self.add_module(str(idx), module)

    def forward(self, X):
        for module in self.children():
            X = module(X)
        return X

In the `__init__` method, we add every module
by calling the `add_modules` method. These modules can be accessed by the `children` method at a later date.
In this way the system knows the added modules,
and it will properly initialize each module's parameters.


When our `MySequential`'s forward propagation method is invoked,
each added module is executed
in the order in which they were added.
We can now reimplement an MLP
using our `MySequential` class.


In [ ]:
net = MySequential(nn.LazyLinear(256), nn.ReLU(), nn.LazyLinear(10))
net(X).shape

torch.Size([2, 10])

Note that this use of `MySequential`
is identical to the code we previously wrote
for the `Sequential` class
(as described in :numref:`sec_mlp`).


## [**Executing Code in the Forward Propagation Method**]

The `Sequential` class makes model construction easy,
allowing us to assemble new architectures
without having to define our own class.
However, not all architectures are simple daisy chains.
When greater flexibility is required,
we will want to define our own blocks.
For example, we might want to execute
Python's control flow within the forward propagation method.
Moreover, we might want to perform
arbitrary mathematical operations,
not simply relying on predefined neural network layers.

You may have noticed that until now,
all of the operations in our networks
have acted upon our network's activations
and its parameters.
Sometimes, however, we might want to
incorporate terms
that are neither the result of previous layers
nor updatable parameters.
We call these *constant parameters*.
Say for example that we want a layer
that calculates the function
$f(\mathbf{x},\mathbf{w}) = c \cdot \mathbf{w}^\top \mathbf{x}$,
where $\mathbf{x}$ is the input, $\mathbf{w}$ is our parameter,
and $c$ is some specified constant
that is not updated during optimization.
So we implement a `FixedHiddenMLP` class as follows.


In [7]:
class FixedHiddenMLP(nn.Module):
    def __init__(self):
        super().__init__()
        # Random weight parameters that will not compute gradients and
        # therefore keep constant during training
        self.rand_weight = torch.rand((20, 20))
        self.linear = nn.LazyLinear(20)

    def forward(self, X):
        X = self.linear(X)
        X = F.relu(X @ self.rand_weight + 1)
        # Reuse the fully connected layer. This is equivalent to sharing
        # parameters with two fully connected layers
        X = self.linear(X)
        # Control flow
        while X.abs().sum() > 1:
            X /= 2
        return X.sum()

In this model,
we implement a hidden layer whose weights
(`self.rand_weight`) are initialized randomly
at instantiation and are thereafter constant.
This weight is not a model parameter
and thus it is never updated by backpropagation.
The network then passes the output of this "fixed" layer
through a fully connected layer.

Note that before returning the output,
our model did something unusual.
We ran a while-loop, testing
on the condition its $\ell_1$ norm is larger than $1$,
and dividing our output vector by $2$
until it satisfied the condition.
Finally, we returned the sum of the entries in `X`.
To our knowledge, no standard neural network
performs this operation.
Note that this particular operation may not be useful
in any real-world task.
Our point is only to show you how to integrate
arbitrary code into the flow of your
neural network computations.


In [8]:
net = FixedHiddenMLP()
net(X)

tensor(0.1312, grad_fn=<SumBackward0>)

We can [**mix and match various
ways of assembling modules together.**]
In the following example, we nest modules
in some creative ways.


In [9]:
class NestMLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.net = nn.Sequential(nn.LazyLinear(64), nn.ReLU(),
                                 nn.LazyLinear(32), nn.ReLU())
        self.linear = nn.LazyLinear(16)

    def forward(self, X):
        return self.linear(self.net(X))

chimera = nn.Sequential(NestMLP(), nn.LazyLinear(20), FixedHiddenMLP())
chimera(X)

tensor(-0.2207, grad_fn=<SumBackward0>)

## Summary

Individual layers can be modules.
Many layers can comprise a module.
Many modules can comprise a module.

A module can contain code.
Modules take care of lots of housekeeping, including parameter initialization and backpropagation.
Sequential concatenations of layers and modules are handled by the `Sequential` module.


## Exercises

1. What kinds of problems will occur if you change `MySequential` to store modules in a Python list?
1. Implement a module that takes two modules as an argument, say `net1` and `net2` and returns the concatenated output of both networks in the forward propagation. This is also called a *parallel module*.
1. Assume that you want to concatenate multiple instances of the same network. Implement a factory function that generates multiple instances of the same module and build a larger network from it.


[Discussions](https://discuss.d2l.ai/t/55)


In [14]:
net = nn.Sequential(NestMLP())

X = torch.rand(1,20)
net(X)
net[0].state_dict()

OrderedDict([('net.0.weight',
              tensor([[ 0.0823, -0.0290, -0.1844,  ..., -0.1236, -0.1741,  0.1223],
                      [ 0.1067, -0.0965, -0.1310,  ..., -0.1205, -0.0400, -0.1095],
                      [ 0.1613, -0.0419, -0.1104,  ..., -0.0629,  0.1193,  0.1382],
                      ...,
                      [ 0.0259, -0.1120, -0.1835,  ...,  0.2110,  0.1469,  0.0937],
                      [-0.1565, -0.0784, -0.0107,  ..., -0.0409, -0.2077,  0.0359],
                      [-0.1174,  0.1220, -0.1856,  ..., -0.1956,  0.1038,  0.2078]])),
             ('net.0.bias',
              tensor([-0.0509, -0.1509,  0.1859,  0.1441,  0.0762,  0.1518, -0.1310, -0.1827,
                      -0.1084, -0.0561,  0.0892, -0.0867,  0.0065, -0.1197, -0.0368, -0.1504,
                       0.2110,  0.1367,  0.1344,  0.0670, -0.0737, -0.0089,  0.0644,  0.1614,
                      -0.1854,  0.1559, -0.0249,  0.1437,  0.0562,  0.1975, -0.1577,  0.1546,
                       0.0709,  

In [20]:
import torch.optim as optim
import torch.nn as nn

class SharedMLP(nn.Module):
  def __init__(self):
    super().__init__()
    self.shared_layer = nn.Linear(10, 10)
    self.layer2 = nn.Linear(10, 5)
    self.out = nn.Linear(5, 1)

  def forward(self, X):
    x1 = torch.relu(self.shared_layer(X))
    x2 = torch.relu(self.shared_layer(X))
    x = x1 + x2
    x = torch.relu(self.layer2(x))
    return self.out(x)


model = SharedMLP()
optimizer = optim.Adam(model.parameters(), lr=0.01)
criterion = nn.MSELoss()

# 8 samples, 10 input dimensions
X = torch.randn(8, 10)

# 8 samples, 1 input dimension
y = torch.randn(8, 1)

# Train
for step in range(1, 4):
  print(f"\n=== Training Step {step} ===")

  print("Shared layer before each step:")
  print(model.shared_layer.weight.data)

  # forward
  y_pred = model(X)
  loss = criterion(y_pred, y)

  # zero gradients
  optimizer.zero_grad()

  # backward
  loss.backward()

  print("Shared layer after backward:")
  print(model.shared_layer.weight.data)

  # update parameters after optimized
  optimizer.step()

  print("Shared layer after update:")
  print(model.shared_layer.weight.data)


=== Training Step 1 ===
Shared layer before each step:
tensor([[ 0.1174,  0.1114,  0.2197, -0.3061, -0.2333, -0.0095,  0.1000, -0.2867,
          0.0539,  0.2322],
        [ 0.1547, -0.2423, -0.2387, -0.2987, -0.2466, -0.2592, -0.0574,  0.0627,
          0.2664, -0.1678],
        [ 0.0539,  0.1513, -0.2161,  0.2679, -0.3066,  0.0686, -0.2788, -0.2205,
          0.2435,  0.0890],
        [ 0.2612, -0.0345,  0.0868, -0.1055,  0.2561, -0.2040, -0.2920,  0.0406,
         -0.0632,  0.1218],
        [ 0.0901,  0.0466,  0.1387,  0.1517, -0.2028,  0.1609, -0.1430,  0.1903,
         -0.2775,  0.1589],
        [ 0.2446,  0.0532, -0.1315, -0.2600,  0.0624, -0.1141,  0.1765,  0.2439,
          0.1431,  0.1055],
        [-0.2912, -0.0134,  0.2953,  0.0291, -0.0337,  0.1042,  0.0112,  0.0819,
         -0.0033,  0.2975],
        [ 0.0051,  0.0344, -0.1542, -0.1350,  0.2983,  0.1130, -0.0870,  0.0545,
          0.3158, -0.1441],
        [-0.2465,  0.2496,  0.3061, -0.3154, -0.3009, -0.0927,  0.1542, 